# 90-day COVID Diagnosis Prediction with Synthea
---

By the end of this tutorial, you should be able to create a patient cohort, build a CNN model with Tensorflow, and perform SageMaker batch transform.

We use Synthea data (synthetic) data to demonstrate the modeling technique. However, readers can choose your own data sets to try. 

## Contents

1. [Background](#1_Background)
1. [Setup](#2_Setup)
1. [Data](#3_Data)
    1. [Citation](#3_A_Citation)
    1. [Cohort Creation](#3_B_CohortCreation)
    1. [Data Cleaning](#3_C_DataCleaning)
1. [Train](#4_Train)
1. [Testing](#5_Testing)
1. [Evaluation](#6_Evaluation)
1. [Extensions](#7_Extensions)

## 1_Background
In this tutorial, you will see an end-to-end example of how to prepare a open dataset and build a convolutional neural network model to predict patient outcomes.

Healthcare data can be challenging to work with and AWS customers have been looking for solutions to solve certain business challenges with the help of data and machine learning (ML) techniques. Some of the data is structured, such as birthday, gender, and marital status, but most of the data is unstructured, such as diagnosis codes or physician’s notes. This data is designed for human beings to understand, but not for computers to comprehend. The key challenges of using healthcare data are as follows:

* How to effectively use both structured and unstructured data to get a complete view of the data
* How to intuitively interpret the prediction results

With the rise of AI/ML technologies, solving these challenges became possible.


## 2_Setup

_This notebook was created and tested on an ml.m5.2xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training, validation and testing data, as well as a bucket to store Athena intermediate results.  This should be within the same region as the Notebook Instance.
- The IAM role arn used to give training and hosting access to your data. The `get_execution_role()` function will return the role you configured for this notebook instance. As we will use both Glue catalog and S3, make sure you configured these access permissions in the role. 

In [ ]:
! pip install -r requirements.txt

In [ ]:
bucket = 'pop-modeling-<your-account-id>'
prefix = 'sagemaker/synthea'

RANDOM_STATE = 2021

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role
import sagemaker
sagemaker_session = sagemaker.Session()

role = get_execution_role()
s3 = boto3.resource('s3')
my_session = boto3.session.Session()

Next, we'll import the Python libraries we'll need for the remainder of the tutorial.

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import sys
import sagemaker
import boto3
from botocore.client import ClientError
from sagemaker.predictor import csv_serializer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_curve, auc
from time import strftime, gmtime
from matplotlib import pyplot as plt

---
## 3_Data


## 3_A_Citation

SyntheaTM is a Synthetic Patient Population Simulator. The goal is to output synthetic, realistic (but not real), patient data and associated health records in a variety of formats.

Copyright 2017-2021 The MITRE Corporation

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## 3_B_CohortCreation
After you get access to the dataset, you can read and issue SQL queries via Athena. You can also save the results as a CSV file for feature use. 

In order to get a seamless experience, we will be using PyAthena library, and issue the queries directly from SageMaker notebook. If you have not installed PyAthena, simple do `!pip install pyathena`

Now, we have successfully created a table called `pop_main`. Next, we will read the data from the created table, and prepare the data for modeling.

In [ ]:
from pyathena import connect
s3_staging_dir='s3://'+bucket+'/athena/temp'
conn = connect(s3_staging_dir=s3_staging_dir, region_name=my_session.region_name)
query_get_all = """SELECT * FROM healthlake_synthea.pop_main"""
data = pd.read_sql_query(query_get_all, conn)
conn.close()

In [ ]:
from ml.data_augument import process_hl_data
hl_data = process_hl_data('data')
hl_data.shape

In [ ]:
data['events'] = data['cond_cd'].str[1:-1].str.replace(',', '').replace('null', '').str.strip() + ' ' +\
                data['proc_cd'].str[1:-1].str.replace(',', '').replace('null', '').str.strip() + ' ' +\
                data['rxnorm'].str[1:-1].str.replace(',', '').replace('null', '').str.strip() + ' ' +\
                data['loinc_cd'].str[1:-1].str.replace(',', '').replace('null', '').str.strip()
target_col='covid90'
data = data[['patient_id', 'age', 'gender', 'if_married', 'events', target_col]].dropna()

In [ ]:
df = pd.merge(data, hl_data, how='left', on='patient_id')
df['events'] = df['events']+df['code_value'].fillna('').str[1:-1].str.replace(',', '').str.strip()
df = df.drop(['code_value', 'code_description'], axis=1).dropna()
df['events'] = df['events'].apply(lambda x: ' '.join(x.split()))
df = df[df['events']!=''].reset_index(drop=True)

## 3_C_DataCleaning 

The final dataset contains 455 records from 45 unique sample patients.

The last attribute, `covid90`, is the target attribute–the attribute that we want the ML model to predict.  Because the target attribute is binary, our model will be performing binary prediction (yes/no), also known as binary classification. 

### Train-Test Split

In [ ]:
# Each patient has more than 1 record
df.shape[0]==df.patient_id.nunique()

In [ ]:
ids = df['patient_id'].drop_duplicates().reset_index(drop=True)
# Split on patient_id
train_id, test_id = np.split(ids.sample(frac=1, random_state=RANDOM_STATE), [int(0.8*len(ids))])

train_id.to_csv('data/train_id.csv', index=False)
test_id.to_csv('data/test_id.csv', index=False)
train_df = df.loc[df['patient_id'].isin(train_id), :]
test_df = df.loc[df['patient_id'].isin(test_id), :]
train_df.to_csv('data/train_df.csv', index=False)
test_df.to_csv('data/test_df.csv', index=False)

Upload the training/testing id to S3

In [ ]:
sagemaker_session.upload_data(path='data/train_id.csv', bucket=bucket, key_prefix=prefix)
sagemaker_session.upload_data(path='data/test_id.csv', bucket=bucket, key_prefix=prefix)
sagemaker_session.upload_data(path='data/train_df.csv', bucket=bucket, key_prefix=prefix)
sagemaker_session.upload_data(path='data/test_df.csv', bucket=bucket, key_prefix=prefix)

### Emedding
Hyperparameters that we can specify during the W2V training:
- `sg` - Training algorithm: 1 for skip-gram; otherwise CBOW.
- `size` - Dimensionality of the word vectors.
- `window` – Maximum distance between the current and predicted word within a sentence.
- `min_count` – Ignores all words with total frequency lower than this.
- `workers` – Use these many worker threads to train the model.
- `iter` – Number of epochs over the corpus.
- `alpha` - Initial learning rate.

In [ ]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
import sys
sys.path.append('ml')
from data import EpochLogger


event_list = [i.split(' ') for i in train_df['events'].values]
event_list = [i  for i in event_list if i!=['']]
epoch_logger = EpochLogger()

EMBEDDING_DIM = 100
WINDOW = 100
EPOCHS = 10
model = Word2Vec(
    event_list,
    sg       = 1,
    vector_size     = EMBEDDING_DIM,
    window   = WINDOW,
    min_count= 1,
    workers  = 2, 
    alpha    = 0.05,
    epochs     = EPOCHS,
    callbacks=[epoch_logger]
)

In [ ]:
model.save('data/W2V_event_dim' + str(EMBEDDING_DIM) + '_win' + str(WINDOW) + '.model')
sagemaker_session.upload_data(path='data/W2V_event_dim' + str(EMBEDDING_DIM) + '_win' + str(WINDOW) + '.model', bucket=bucket, key_prefix='{}/{}'.format(prefix,'sagemaker/train'))

### PreProcessing
Scale, Imputation, convert to embeddings

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('ml')

# Import custom lib
from data import read_csv_s3, load_embedding_matrix_trained_model, EpochLogger, df2tensor, get_csv_output_from_s3
from evaluation import plot_metrics, plot_test_auc
from evaluation_tf import f1, METRICS, early_stopping
from config import RANDOM_STATE, MAX_LENGTH, EPOCHS, BATCH_SIZE, EMBEDDING_DIM,  filters, kernel_size

# Import tensorflow lib
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Conv1D, GlobalMaxPool1D, concatenate, MaxPooling1D, Lambda
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import backend as k
from tensorflow.keras.optimizers import Adam

# Import pandas, numpy, plot
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib as mpl
from matplotlib import pyplot as plt

# Import sklearn lib
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
embedding_matrix, idx_to_code_map, code_to_idx_map, unkown_idx = load_embedding_matrix_trained_model(
    model_name='data/W2V_event_dim100_win100.model', embedding_dim=EMBEDDING_DIM)

labels = df[target_col]
feature_static = df[['age', 'gender', 'if_married']]
static_feature_names = [i for i in feature_static.columns]
STATIC_F_DIM = feature_static.shape[1]

indices_train = df['patient_id'].isin(train_id)
indices_test = df['patient_id'].isin(test_id)
train_df = df[indices_train].reset_index(drop=True)
y_train = labels[indices_train].reset_index(drop=True)
test_df = df[indices_test].reset_index(drop=True)
y_test = labels[indices_test].reset_index(drop=True)

static_train = feature_static.loc[indices_train, :]
static_test =  feature_static.loc[indices_test, :]

# Impute median for static features
pipe = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
pipe.fit(static_train)
static_train_imputed = pipe.transform(static_train)
static_test_imputed = pipe.transform(static_test)

train_processed_df = pd.concat([pd.DataFrame(static_train_imputed, columns=feature_static.columns), 
                                train_df['events'], train_df[target_col]], axis=1)
test_processed_df  = pd.concat([pd.DataFrame(static_test_imputed, columns=feature_static.columns), 
                                test_df['events'], test_df[target_col]], axis=1)

In [ ]:
df_train, df_validation = train_test_split(train_processed_df, test_size=0.2, random_state=RANDOM_STATE)
df_test = test_processed_df
df_train.to_csv('data/train.csv', index=False)
df_train[static_feature_names].to_csv('data/train-static.csv', index=False)
df_validation.to_csv('data/validation.csv', index=False)
df_validation[static_feature_names].to_csv('data/valid-static.csv', index=False)
df_test.to_csv('data/test.csv', index=False)
df_test[static_feature_names].to_csv('data/test-static.csv', index=False)

train_input = sagemaker_session.upload_data(path='data/train.csv', bucket=bucket, key_prefix='{}/{}/{}'.format(prefix, 'sagemaker','train'))
train_static_input = sagemaker_session.upload_data(path='data/train-static.csv', bucket=bucket,key_prefix='{}/{}/{}'.format(prefix, 'sagemaker','train'))
valid_input = sagemaker_session.upload_data(path='data/validation.csv', bucket=bucket,key_prefix='{}/{}/{}'.format(prefix, 'sagemaker','validation'))
valid_static_input = sagemaker_session.upload_data(path='data/valid-static.csv', bucket=bucket,key_prefix='{}/{}/{}'.format(prefix, 'sagemaker','validation'))
test_input = sagemaker_session.upload_data(path='data/test.csv', bucket=bucket, key_prefix='{}/{}/{}'.format(prefix, 'sagemaker','test'))
test_static_input = sagemaker_session.upload_data(path='data/test-static.csv', bucket=bucket, key_prefix='{}/{}/{}'.format(prefix, 'sagemaker','test'))

---
## 4_Train

Let's take the preprocessed training data and fit a customized TensorFlow Model. Sagemaker provides prebuilt TensorFlow containers that can be used with the Python SDK. The previous Scikit-learn job preprocessed the raw input data and made it ready for machine learning models. Now, we call the `tf_model.py` script and use the transformed data to train a model.

Some common parameters we need to configer here are:
* `entry_point`: defines the entry point file name
* `source_dir`: defines the directory where entry point file and requirements.txt file are
* `role`: Role ARN
* `train_instance_count`: number of training instance for the job 
* `train_instance_type`: type of training instance. For large dataset with deep learning, GPU enabled instances can significantly improve the training speed
* `code_location`: the location of sourdir.tar.gz file that contains the scripts and configurations
* `output_path`: the location of model artifact model.tar.gz
* `framework_version`: the TensorFlow framework version
* `py_version`: python version
* `script_mode`: if you want to use script model for the training job
* `base_job_name`: for SageMaker to group your training jobs with this prefix and have more organized report on SageMaker console
* `hyperparameters`: hyperparameters that you want to pass into the model
* `metric_definitions`: metrics that you want to emit to SageMaker console and further to CloudWatch

In [ ]:
from sagemaker.tensorflow import TensorFlow
tf_estimator = TensorFlow(entry_point='tf_model.py', 
                          source_dir='ml',
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m5.xlarge',
                          code_location='s3://{}/{}/{}'.format(bucket, prefix, 'sagemaker/tf-model-script-mode'),
                          output_path='s3://{}/{}/{}'.format(bucket, prefix, 'sagemaker/output'),
                          framework_version='1.15', 
                          py_version='py3',
                          script_mode=True,
                          base_job_name='POP-CNN',
                          hyperparameters={'epochs': 10,
                                           'learning-rate': 1e-3,
                                           'batch-size':32, 
                                           'max-length':MAX_LENGTH,
                                           'static-f-dim': len(static_feature_names),
                                           'target-col': target_col},
                          metric_definitions=[{
                                         "Name": "f1",
                                         "Regex": "val_f1_m: (.*?)$"
                                     },{
                                         "Name": "loss",
                                         "Regex": "val_loss: (.*?) -"
                                     },{
                                         "Name": "precision",
                                         "Regex": "val_precision: (.*?) -"
                                     },{
                                         "Name": "recall",
                                         "Regex": "val_recall: (.*?) -"
                                     }]
                         )
tf_estimator.fit({'training': f's3://{bucket}/{prefix}/sagemaker/train',
                  'validation': f's3://{bucket}/{prefix}/sagemaker/validation'})

---
## 5_Testing

Let's prepare the test data as the data format accepted by our model, and upload it to s3 for future use. 

In [ ]:
input_tensors = df2tensor(test_input, test_static_input, maxlen=MAX_LENGTH, model_name='data/W2V_event_dim100_win100.model')
with open("data/test.jsonl", "w") as text_file:
    text_file.write(input_tensors)
test_s3 = sagemaker_session.upload_data(path='data/test.jsonl', bucket=bucket, key_prefix='{}/{}/{}'.format(prefix, 'sagemaker','test'))

---
## 5_BatchTransform

We leverage SageMaker batch transform to do a batched prediction. 

In [ ]:
tf_transformer = tf_estimator.transformer(
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    assemble_with='Line',
    accept='text/csv',
    output_path='s3://{}/{}/{}'.format(bucket, prefix, 'sagemaker/batch-transform'), 
    max_payload=100)

# Make batch transform on test input
tf_transformer.transform(test_s3, content_type='application/jsonlines')
print("Waiting for transform job: " + tf_transformer.latest_transform_job.job_name)
tf_transformer.wait()

## 6_Evaluation

Please keep in mind, the Synthea data is artificially synthesized. The modeling results are only provided as a general evaluation step, and should not be the focus here.

In [ ]:
output = get_csv_output_from_s3(tf_transformer.output_path, 'test.jsonl.out')
y_prob = np.array(output).squeeze()
y_pred = (y_prob>0.5).astype(int)
test_df = read_csv_s3(bucket, 'sagemaker/synthea/sagemaker/test', 'test.csv')
y_test = test_df['covid90'].values

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
plot_test_auc(y_test, y_prob)